In [ ]:
from pykml import parser
import pandas as pd
import numpy as np
import requests
# Parse the KML file
with open('D74.kml', 'r') as kml_file:
    kml_doc = parser.parse(kml_file).getroot()
altitudeMode = kml_doc.Document.Placemark.getchildren()[2].getchildren()[0]


attributes = kml_doc.findall(

    ".//{http://www.google.com/kml/ext/2.2}SimpleArrayData")

times = list(kml_doc.getchildren()[0].Placemark.getchildren()[2].findall(
    ".//{http://www.opengis.net/kml/2.2}when"))
coords = list(kml_doc.getchildren()[0].Placemark.getchildren()[2].findall(
    ".//{http://www.google.com/kml/ext/2.2}coord"))
print(len(times), " times loaded")
print(len(coords), " coordinates loaded")
acc_horiz = []
acc_vert = []
course = []
speed_kts = []
altitude = []
bank = []
pitch = []
for param in attributes:
    if param.get("name") == 'acc_horiz':
        for attr in param.getchildren():
            acc_horiz.append(attr.text)
    elif param.get("name") == 'acc_vert':
        for attr in param.getchildren():
            acc_vert.append(attr.text)
    elif param.get("name") == 'course':
        for attr in param.getchildren():
            course.append(attr.text)
    elif param.get("name") == 'speed_kts':
        for attr in param.getchildren():
            speed_kts.append(attr.text)
    elif param.get("name") == 'altitude':
        for attr in param.getchildren():
            altitude.append(attr.text)
    elif param.get("name") == 'bank':
        for attr in param.getchildren():
            bank.append(attr.text)
    elif param.get("name") == 'pitch':
        for attr in param.getchildren():
            pitch.append(attr.text)


times = [str(t) for t in times]
coords = [str(c) for c in coords]

print("Altitude Mode:", altitudeMode)
print(len(bank))
print(len(pitch))
print(len(altitude))
print(len(speed_kts))
print(len(course))
print(len(acc_vert))
print(len(acc_horiz))


In [ ]:
data = [times, coords, bank, pitch, altitude, speed_kts, course, acc_horiz, acc_vert]

# Transpose the array using the zip() function
data = [list(x) for x in zip(*data)]

data = pd.DataFrame(data, columns =  ['times', 'coords', 'bank', 'pitch', 'altitude', 'speed_kts', 'course', 'acc_horiz', 'acc_vert'])

data[['longitude', 'latitude', 'height']] = data['coords'].str.split(' ', expand=True)

data.drop('coords', axis=1, inplace=True)

data['timestamp'] = pd.to_datetime(data['times'])
data['bank'] = data['bank'].astype(float)
data['pitch'] = data['pitch'].astype(float)
data['altitude'] = data['altitude'].astype(float)
data['speed_kts'] = data['speed_kts'].astype(float)
data['course'] = data['course'].astype(float)

data['latitude'] = data['latitude'].astype(float)
data['longitude'] = data['longitude'].astype(float)


data['height'] = data['height'].astype(float)

for i in range(len(data["height"])):
    if data["height"][i] < 0:
        data["height"][i] = 0

data['acc_horiz'] = data['acc_horiz'].astype(int)
data['acc_vert'] = data['acc_vert'].astype(int)

data.set_index('timestamp', inplace=True)
data

In [ ]:
import matplotlib.pyplot as plt
# data['altitude'] = data['altitude'].interpolate(method='linear')
# Create a figure and axis object
fig, ax1 = plt.subplots(figsize=(20, 8))

# Plot the 'altitude' column on the first y-axis
ax1.plot(data['altitude'], color='blue')
ax1.set_ylabel('Altitude (ft)', color='blue')

# Create a second y-axis object
ax2 = ax1.twinx()

# # Plot the 'speed_kts' column on the second y-axis
ax2.plot(data['speed_kts'], color='red')
ax2.set_ylabel('Speed (kts)', color='red')

# Show the plot
plt.show()

In [ ]:
for i in range(6060,6150, 5):
    if data['altitude'][i] > 800 and data['altitude'][i] < 1000:
        print('i=',i)
        print(data['altitude'][i])
        print("crs", data['course'][i])
        print("lat" , data['latitude'][i], end=" ")
        print("lon", data['longitude'][i])

In [ ]:
import numpy as np
change = 340 - 0
if abs(change) > 180:
    change = (360 - abs(change)) * np.sign(change) * -1

print(change)

In [ ]:
import numpy as np

count = 0 
last = 0
increasing = True
start = 0

turn_segments = []

for i in range(5,len(data) - 5, 5):
    if data['altitude'][i] > 800 and data['altitude'][i] < 1000:

        change = float(data['course'][i]) - float(data['course'][last])

        # check if the change is greater than 180
        # if it is, then we need to change the sign of the change
        if abs(change) > 180:
            change = (360 - abs(change)) * np.sign(change) * -1
        if change > 0:
            if increasing == False:
                count = 0
                increasing = True
                start = i
                # start = last
            count += 1
        else:
            if increasing == True:
                count = 0
                increasing = False
                start = i
            count += 1
        if count >= 20:
            if abs(data["longitude"][i] - data["longitude"][start]) < 0.01 and abs(data["latitude"][i] - data["latitude"][start]) < 0.01:
                turn_segments.append([start, i])
            count = 0
            start = i
        last = i
    

turn_segments   


In [ ]:
import requests

def get_elevations():
    elevations = []
    queue = coords
    # print(queue)
    # while len(queue) > 0:

    while len(queue) > 0:
        print(queue)
        print(len(queue))
        start = True
        request = queue[:100]
        locations = ''
        for coord in request:
            lat = coord.split(" ")[1]
            lon = coord.split(" ")[0]
            if not start:
                locations = locations + f'|{lat},{lon}'
            else:
                locations = locations + f'{lat},{lon}'
                start = False
            queue.pop(0)
        print(locations)

        base_url = "https://api.opentopodata.org/v1/ned10m"
        params = {
        'locations': locations,
        'format': 'json'
        }

        try:
            response = requests.get(base_url, params=params)
            data = response.json()
        # print(data)
        # print(response.status_code)
        # Check if the request was successful
            if response.status_code == 200 and 'results' in data:
                for elev in data['results']:
                    elevations.append(float(elev['elevation'] * 3.24))
            else:
                print(f'Error: {data["error"]["message"]}')
        except Exception as e:
            print(f'Error: {str(e)}')
    print(len(elevations))
    return elevations

# elevations = get_elevations()
# print(elevations)

In [ ]:
segments = []
start = 0
end = 0
takeoffs = 0
landings = 0
# currentlyAirborne = data['height'][0] > 75
# lastAirborne = True
# count = 0
i = 0
count = 0
while i < len(data):
    count += 1
    if (data['height'][i] > 500):
        i += 60
        continue

    if (data['height'][i] < 25):
        if i+15 >= len(data):
            end = data['speed_kts'][len(data)-1]
        else:
            end = data['speed_kts'][i+15]
        start = data['speed_kts'][i]
        if data['height'][i] < 25:
            if end - start > 25 and start < 25 and 50 < end < 100:
                segments.append({"start":i, "end": i + 15,   "status": "takeoff", "score" : "NA"})
                takeoffs += 1
                i += 15
                continue
    i += 5
print(count/len(data))
i = 0
count = 0
while i < len(data):
    count += 1
    if (data['height'][i] > 500):
        i += 60
        continue

    if (data['height'][i] < 25):
        if i+20 >= len(data):
            end = data['speed_kts'][len(data)-1]
        else:
            end = data['speed_kts'][i+20]
        start = data['speed_kts'][i]

        if (start - end > 15 and start < 100 and end < 25 ):
            segments.append({"start":i, "end": i + 15,   "status": "landing" ,"score" : "NA"})
            landings += 1
            i += 60
            continue
    i += 5

print(count/len(data))

# sort segments by starting point. 
segments.sort(key=lambda x: x["start"])

startPoint = 0
i = 0
while i < len(segments):
    # just tookoff, last segment was taxi
    if segments[i]["status"] == "takeoff":
        segments.insert(i, {"start":startPoint, "end": segments[i]["start"],   "status": "taxi", "score" : "NA"})
        startPoint = segments[i + 1]["end"]
    # just landed, last segment was airborne
    elif segments[i]["status"] == "landing":
        segments.insert(i, {"start":startPoint, "end": segments[i]["start"],   "status": "airborne", "score" : "NA"})
        startPoint = segments[i + 1]["end"]
    i += 2

if len(segments) != 0:
    segments.append({"start":segments[-1]["end"], "end":len(data),   "status": "taxi", "score" : "NA"})
else:
    segments.append({"start":0, "end":len(data),   "status": "airborne", "score" : "NA"})

print(segments)

In [ ]:
### Scoring takeoffs and landings

scoreArr = []

# calculate score for segments that are out of bounds.
def calculateScore (diff):
    score = 100 * np.exp(-0.12 * diff)
    return score
print(len(data))
for i in range(0, len(segments), 1):
        score = 100
        if segments[i]['status'] in ("takeoff", "landing"):
                avg = sum(data["course"][segments[i]["start"]:segments[i]["end"]])/(segments[i]["end"]-segments[i]["start"])
                for j in range(segments[i]['start'], segments[i]['end'], 1):
                        course = data['course'][j]
                        curScore = calculateScore(abs(course - avg))
                        score = (score + curScore) / 2
                # scoreArr.append(score)
                segments[i]['score'] = str(score)
        if (segments[i]['status'] not in ("airborne")):
                data.drop(data.index[segments[i]['start']:segments[i]["end"]], inplace=True)

print(len(data))
print(scoreArr)
print(len(scoreArr))

In [ ]:
# touch and go

def touch_and_go (start, end):
    if ((data["height"][start] > 30) and (data["height"][end] > 30) and (data["height"][(end-start)//2+start] < 25)):
        return 1
    
touch_and_go_segments = []
i = 0
while i < len(data):
    if (i < len(data)-40):
        if (touch_and_go(i, i+40)):
            touch_and_go_segments.append({"start": i,"end": i+40,"status": "touch&go","score": "NA"})
            i += 40
            continue

    i += 1
i = 0
print(touch_and_go_segments)

In [ ]:
# score touch and go

def calculateScore (diff):
    score = 100 * np.exp(-0.12 * diff)
    return score
print(len(data))
for i in range(0, len(touch_and_go_segments), 1):
        score = 100
        if touch_and_go_segments[i]['status'] in ("touch&go"):
                avg = sum(data["course"][touch_and_go_segments[i]["start"]:touch_and_go_segments[i]["end"]])/(touch_and_go_segments[i]["end"]-touch_and_go_segments[i]["start"])
                for j in range(touch_and_go_segments[i]['start'], touch_and_go_segments[i]['end'], 1):
                        course = data['course'][j]
                        curScore = calculateScore(abs(course - avg))
                        score = (score + curScore) / 2
                # scoreArr.append(score)
                touch_and_go_segments[i]['score'] = str(score)
        if (touch_and_go_segments[i]['status'] not in ("airborne")):
                data.drop(data.index[touch_and_go_segments[i]['start']:touch_and_go_segments[i]["end"]], inplace=True)

print(touch_and_go_segments)

In [ ]:
# Stall speed
Vso = 41
start = -1
last = -1
slow_flight_segments = []
for i in range(0, len(data), 50):
    # elev = elevations[i]
    difference = abs(data["altitude"][i])
    # print(difference)
    ground_speed = data['speed_kts'][i]
    slow_flt = Vso + 10
    near_stall_speed = abs(ground_speed - slow_flt)

    # if we have been long enough since our last data point, end the segment. 
    if (i - last > 100 or i >= len(data) - 100) and start != -1:
        slow_flight_segments.append([start, last])
        start = -1

    if difference >= 1500 and near_stall_speed <= 10:
        if start == -1:
            start = i
        last = i
        print('idx ' , i, ground_speed, " kts ground speed " , data['latitude'][i], data['longitude'][i])


print(slow_flight_segments)   


for slow_segment in slow_flight_segments:
    # get index of splice, put it in at the end. 
    index = -1
    for i in range(len(segments)):
        if segments[i]['end'] >= slow_segment[0]:
            index = i
            break
        else:
            continue

    if index != -1:
        # if the overall segment is greater than ours, then we need to split it.
        if segments[index]['end'] > slow_segment[1]:
            segments.insert(index + 1, {"start":slow_segment[1], "end":segments[index]["end"], "status": segments[index]["status"]})
        else:
            segments[index + 1]["start"] = slow_segment[1]

        segments[index]['end'] = slow_segment[0]
        segments.insert(index + 1, {"start":slow_segment[0], "end":slow_segment[1], "status": "slow_flight"})
        index = -1
# find the segmen t that the start is in, end it early. Find the segment that the end is in, start it late. 
    
print(segments)

In [ ]:
data.plot(x='longitude', y='latitude', kind='scatter', figsize=(20, 8),
          alpha=0.5, c='altitude', cmap=plt.get_cmap('jet'), colorbar=True)


In [ ]:

Vr = 50
Vso = 41
Vs1 = 47
Vx = 59
Vy = 73
Vsw = 50
Vref = 60
Vne = 160
Vno = 128
Vmxt = 10

